In [91]:
base_path = 'C:/Users/samar/OneDrive/Desktop/UVA/Stochastic Hydrology'
src_dir_gas = base_path + '/Data_Final/Gas_Final'
src_dir_coal = base_path + '/Data_Final/Coal_Final'
para_pat = r'\n\n+'
token_pat = r'([\W_]+)'

In [104]:
import glob
import re

import pandas as pd
from numpy.random import randint
import sys; sys.path.append(base_path)
import textman as tx
import sqlite3
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import KernelDensity as KDE
from sklearn.preprocessing import scale

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML

import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\samar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [105]:
%matplotlib inline
sns.set()
plt.style.use('fivethirtyeight')

In [106]:
files = glob.glob(src_dir_coal+'/*.txt')
corpus = pd.DataFrame([f.replace('.txt','').split('/')[-1].split('-') for f in files], 
                    columns=['a','doc_id'])
files_gas = glob.glob(src_dir_gas+'/*.txt')
corpus_gas=pd.DataFrame([f.replace('.txt','').split('/')[-1].split('-') for f in files_gas], 
                    columns=['a','doc_id'])
#corpus['bio_type'] = corpus['bio_type'].map(lambda x: x.lstrip('Bios\\'))
#corpus = corpus[BTYPE]
corpus['doc_content'] = [open(f, 'r', encoding='ISO-8859-1').read() for f in files]
corpus_gas['doc_content'] = [open(f, 'r', encoding='utf-8').read() for f in files_gas]
#stuff = corpus[['subject_type','subject']]
#stuff.to_csv('stuff.csv', index = True, header = True)
#corpus.head()
#files
corpus=corpus.drop(['a'], axis=1)
corpus=corpus.set_index('doc_id')
corpus_gas=corpus_gas.drop(['a'], axis=1)
corpus_gas=corpus_gas.set_index('doc_id')
#corpus
corpus_gas

,doc_content
doc_id,
1,"﻿Hydraulic fracturing, often called fracking, ..."
10,﻿This paper uses the Analytic Hierarchy Proces...
11,"﻿Stalling progress in European, Chinese and La..."
2,﻿Growing attention has been paid to understand...
3,﻿Following the shale gas boom in the United St...
4,﻿Innovation in energy technology is framed as ...
5,﻿Shale gas in the EU and its Member States fac...
6,﻿In consideration of the size and geographic c...
7,﻿The notion of a ‘policy mix’ can describe int...


In [107]:
tokens, vocab = tx.create_tokens_and_vocab(corpus, src_col='doc_content')
tokens['token_num'] = tokens.groupby(['doc_id']).cumcount()
tokens = tokens.reset_index()[['doc_id','token_num','term_id']]
tokens = tokens[tokens.term_id.isin(vocab[vocab.go].index)]
tokens = tokens.set_index(['doc_id','token_num'])

tokens_gas, vocab_gas = tx.create_tokens_and_vocab(corpus_gas, src_col='doc_content')
tokens_gas['token_num'] = tokens_gas.groupby(['doc_id']).cumcount()
tokens_gas = tokens_gas.reset_index()[['doc_id','token_num','term_id']]
tokens_gas = tokens_gas[tokens_gas.term_id.isin(vocab_gas[vocab_gas.go].index)]
tokens_gas = tokens_gas.set_index(['doc_id','token_num'])

In [113]:
tokens_gas

term_id       term_str
doc_id token_num                        
1      0             3371      hydraulic
       1             2989     fracturing
       3             1152         called
       4             2982       fracking
       5             2268       drilling
       6             6670      technique
       7             5213     previously
       8             3483   inaccessible
       9             4687            oil
       10            3058            gas
       11            5761       reserves
       13            5263        process
       15             571            aid
       16            2450         energy
       17            3535   independence
       18            1717  controversial
       19            5390         public
       20            4720        opinion
       21            2218        divided
       22            3204         guided
       23             555         agenda
       24            6092        setting
       25            2997        framing
       26            6471          study
       27            1671        content
       28             660       analyses
       29            4540           news
       30            1793       coverage
       31            2982       fracking
       33            1139          cable
...                   ...            ...
9      6014          6240       socially
       6015          3594       informed
       6016           736       approach
       6017          3145     governance
       6018          2760   facilitating
       6019          6360   stakeholders
       6020          1447    communities
       6021          1897       decision
       6022          4170         making
       6023          5600        reflect
       6024          4086          local
       6025          1678       contexts
       6026           486        address
       6027          4510          needs
       6028          3437       impacted
       6029          1447    communities
       6030           423      according
       6031          5545       recently
       6032          5399      published
       6033          1648   consultation
       6034          4066     literature
       6035           716        appears
       6036          1841        current
       6037          3148     government
       6039          5818         retain
       6040          1241         centre
       6041           736       approach
       6042          6103          shale
       6043          3058            gas
       6044          3145     governance

[42485 rows x 2 columns]

In [114]:
tokens['term_str'] = tokens.term_id.map(vocab.term)
tokens_gas['term_str'] = tokens_gas.term_id.map(vocab_gas.term)

C:\Users\samar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [115]:
vectorizer = TfidfVectorizer(use_idf=1, stop_words='english', token_pattern=r'[A-Za-z][A-Za-z][A-Za-z]+')
X = vectorizer.fit_transform(corpus.doc_content.values.tolist())
X_gas = vectorizer.fit_transform(corpus_gas.doc_content.values.tolist())
v = pd.DataFrame(vectorizer.get_feature_names(), columns=['term_str'])
v['idf'] = vectorizer.idf_
#v


In [116]:
#cutoff = 0.5
#v = v[v.idf > cutoff].sort_values('idf', ascending=False).sample(1000)
my_v = v.term_str.tolist()
#v=v[v.idf > cutoff]
#v
my_v=v.term_str.tolist()
#my_v

In [117]:
tokens = tokens[tokens.term_str.isin(my_v)]
corpus = tx.gather_tokens(tokens, level=0, col='term_str')\
    .reset_index().rename(columns={'term_str':'doc_content'})

tokens_gas = tokens_gas[tokens_gas.term_str.isin(my_v)]
corpus_gas = tx.gather_tokens(tokens_gas, level=0, col='term_str')\
    .reset_index().rename(columns={'term_str':'doc_content'})
#corpus_gas.head()
corpus.head()

,doc_id,doc_content
0,1,article opposition country substantial coal pr...
1,10,engaging coal gas industry complex risks faced...
2,11,planning build series coal fired power plants ...
3,2,article power structure research approach expl...
4,3,coal industry experienced economic decline pas...


In [118]:
#corpus = tx.gather_tokens(tokens, level=0, col='term_str')\
   # .reset_index().rename(columns={'term_str':'doc_content'})
corpus['doc_label'] = corpus.apply(lambda x: "Coal-{}".format(x.doc_id), 1)
corpus_gas['doc_label'] = corpus_gas.apply(lambda x: "Gas-{}".format(x.doc_id), 1)

In [119]:
corpus_gas

,doc_id,doc_content,doc_label
0,1,hydraulic fracturing called fracking drilling ...,Gas-1
1,10,paper uses analytic hierarchy process ahp sele...,Gas-10
2,11,progress european chinese american shale attri...,Gas-11
3,2,growing attention paid understanding public ri...,Gas-2
4,3,following shale gas boom united states unconve...,Gas-3
5,4,innovation energy technology framed potential ...,Gas-4
6,5,shale gas member states faces increasing atten...,Gas-5
7,6,consideration size geographic concentration pr...,Gas-6
8,7,notion policy mix interactions wide range inno...,Gas-7
9,8,paper examines shale gas development situation...,Gas-8


In [120]:
corpus[['doc_label','doc_content']].to_csv('coal-corpus.csv', index=False)
corpus_gas[['doc_label','doc_content']].to_csv('gas-corpus.csv', index=False)
corpus_add=corpus.append(corpus_gas)
corpus_add[['doc_label','doc_content']].to_csv('coal-gas-corpus.csv', index=False)

In [121]:
#Couldnot run the mallet from the jupyter notebook.
#Hence writing the commands used
#Results will also be submitted with the product
#mallet  import-file --input filename-corpus.csv --output filename-corpus.mallet --keep-sequence TRUE